In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov  2 14:02:02 2019

@author: volka
"""
import sklearn
from textblob import TextBlob
from sklearn import model_selection, preprocessing , linear_model,naive_bayes,metrics
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn import decomposition , ensemble

import pandas , xgboost , numpy , textblob , string
from keras.preprocessing import text , sequence
from keras import layers, models, optimizers 

from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
data8 = pd.read_csv("train.tsv", sep = "\t")












Using TensorFlow backend.


In [2]:
data8["Sentiment"].replace(0, value = "negative" , inplace = True)
data8["Sentiment"].replace(1, value = "negative" , inplace = True)

In [3]:
data8["Sentiment"].replace(3, value = "pozitive" , inplace = True)
data8["Sentiment"].replace(4, value = "pozitive" , inplace = True)

In [4]:
data8 = data8[(data8.Sentiment == "negative") | (data8.Sentiment == "pozitive")]



In [5]:

df = pd.DataFrame()
df["text"] =  data8["Phrase"]
df["label"] = data8["Sentiment"]

In [6]:
#buyuk-kucuk donusumu
df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')
#sayılar
df['text'] = df['text'].str.replace('\d','')
#stopwords
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
#seyreklerin silinmesi
sil = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))
#lemmi
from textblob import Word
#nltk.download('wordnet')
df['text'] = df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


In [7]:
## TEST -TRAIN Create

train_x, test_x, train_y, test_y = model_selection.train_test_split(df["text"],
                                                                   df["label"], 
                                                                    random_state = 1)



In [8]:
encoder = preprocessing.LabelEncoder()

In [9]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [10]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [12]:
x_train_count

<57358x13558 sparse matrix of type '<class 'numpy.int64'>'
	with 282287 stored elements in Compressed Sparse Row format>

In [13]:
vectorizer.get_feature_names()[0:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [14]:
x_train_count.toarray

<bound method _cs_matrix.toarray of <57358x13558 sparse matrix of type '<class 'numpy.int64'>'
	with 282287 stored elements in Compressed Sparse Row format>>

In [15]:
#wordlevel

In [34]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [35]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [36]:
tf_idf_word_vectorizer.get_feature_names()[0:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [37]:
x_test_tf_idf_word.toarray

<bound method _cs_matrix.toarray of <19120x13558 sparse matrix of type '<class 'numpy.float64'>'
	with 93386 stored elements in Compressed Sparse Row format>>

In [26]:

# ngram level tf-idf

In [38]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range = (2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [39]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

In [ ]:
# characters level tf-idf

In [40]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer = "char", ngram_range = (2,3))
tf_idf_chars_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [41]:
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

In [ ]:
###LOGISTIC REGRESSION

In [42]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("Count Vectors Doğruluk Oranı:", accuracy)

Count Vectors Doğruluk Oranı: 0.8366643837166177


In [52]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)

accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

Word-Level TF-IDF Doğruluk Oranı: 0.8331604456468108


In [96]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_ngram, 
                                           test_y, 
                                           cv = 10).mean()

print("N-GRAM TF-IDF Doğruluk Oranı:", accuracy)

N-GRAM TF-IDF Doğruluk Oranı: 0.7481168394370105


In [97]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_chars,train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_tf_idf_chars, 
                                           test_y, 
                                           cv = 10).mean()

print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7817973765716255


In [98]:
### NAİVE BAYES

nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_count , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count vectors accuracy rate : " , accuracy)


Count vectors accuracy rate :  0.8365074526110133


In [ ]:


nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word - Level TF-IDF accuracy rate : " , accuracy)





In [100]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF accuracy rate : " , accuracy)

N-GRAM TF-IDF accuracy rate :  0.7481168394370105


In [99]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_chars , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()
print("CHARLEVEL Doğruluk Oranı:", accuracy)

CHARLEVEL Doğruluk Oranı: 0.7817973765716255


In [101]:
## RANDOM FOREST


rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_count , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_count,
                                           test_y,
                                           cv = 10).mean()

print("Count vectors accuracy rate : " , accuracy)



Count vectors accuracy rate :  0.8365074526110133


In [102]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 10).mean()

print("Word - Level TF-IDF accuracy rate : " , accuracy)

Word - Level TF-IDF accuracy rate :  0.8330558705048776


In [103]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_ngram , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv = 10).mean()

print("N-GRAM TF-IDF accuracy rate : " , accuracy)

N-GRAM TF-IDF accuracy rate :  0.7481168394370105


In [104]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_chars , train_y)
accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_chars,
                                           test_y,
                                           cv = 10).mean()

print("Char level accuracy rate : " , accuracy)

Char level accuracy rate :  0.7817973765716255


In [ ]:
loj_model.predict(" yes i like this film")

In [56]:
yeni_yorum =pd.Series(" the bad movie I have ever had wacthed , so bad , dramatic ")




In [57]:
v = CountVectorizer()
v.fit(train_x)
yeni_yorum = v.transform(yeni_yorum)

In [58]:
loj_model.predict(yeni_yorum)

array([0])